## Reframing api_code.ipynb as a set of functions

In [1]:
import os
from openai import AzureOpenAI
import pandas as pd

In [2]:
class SASAzureOpenAILLM():
    def __init__(self,client = None,endpoint = None, deployment_name= None,api_key = None,api_version = None,
                temperature = 0.7 ,max_tokens = 16,n = 1):
        self.client = client
        self.endpoint = endpoint
        self.deployment_name = deployment_name
        self.api_key = api_key
        self.api_version = api_version
        self.prompt = []
        self.temperature = temperature
        self.max_tokens = max_tokens
        self.n = n

    def set_client(self, endpoint = None, api_key = None, api_version = None):
       if endpoint is None:
          try:
            endpoint = os.environ["AZURE_OPENAI_ENDPOINT"] 
          except KeyError:
            raise ValueError("Endpoint must be provided or set in AZURE_OPENAI_ENDPOINT environment variable")
       if api_key is None:
            try:
                 api_key = os.environ["AZURE_OPENAI_API_KEY"]
            except KeyError:
                 raise ValueError("API key must be provided or set in AZURE_OPENAI_API_KEY environment variable")
       if api_version is None:
          try:
             api_version = os.environ["AZURE_OPENAI_API_VERSION"]
          except KeyError:
             raise ValueError("API version must be provided or set in AZURE_OPENAI_API_VERSION environment variable")   
          
       self.client = AzureOpenAI(api_key = api_key,  api_version = api_version, azure_endpoint = endpoint)

    
    def get_client(self):
        return self.client

    def set_prompt(self, system_prompt = None, user_prompt = None, example = None):
        if system_prompt == None: 
            system_prompt = "You are a helpful assistant. Using the provided context, respond with the answer only."
        if user_prompt == None:
            user_prompt = "Echo the context."
        if example is None:
             self.prompt = [
             {
                 "role": "system",
                 "content": system_prompt
             },
             {
                 "role": "user",
                 "content": f"{user_prompt}\n" 
             }
             ]
        else:
             self.prompt = [
             {
                 "role": "system",
                 "content": system_prompt
             },
             {
                 "role": "user",
                 "content": f"{user_prompt}\nExample(s): {example}\n"
             }
             ]
    
    def get_prompt(self):
        return "".join((self.prompt[0]["content"], self.prompt[1]["content"]))
    
    def get_response(self, context = None, client = None, deployment_name = None, prompt = None, temperature=None, max_tokens=None, n=None, verbose = False):
        if client is None:
            client = self.client
        if deployment_name is None:
            deployment_name = self.deployment_name
        if prompt is None:
            prompt = self.prompt
        if temperature is None:
            temperature = self.temperature
        if max_tokens is None:
            max_tokens = self.max_tokens
        if n is None:
            n = self.n
        # if context is none, then do nothing
        if context is None or len(context) == 0:
            print("No context provided")
            return ""
        else: 
            prompt[1]["content"] = prompt[1]["content"] + f"Context: {context}"
            completion = client.chat.completions.create(
                model = deployment_name,
                messages = prompt,
                temperature = temperature,
                max_tokens = max_tokens
            )
            if verbose:
                return completion.choices[0].message.content
            return completion.choices[0].message.content
        
def reset_indf(): 
    global input_data
    input_data = pd.read_csv("book_desc.csv")
    new_row = {'title': 'The Crash', 'author': "Frieda McFadden", 'description': ''}
    input_data.loc[len(input_data)] = new_row
    
def execute(endpoint=None, api_key=None, api_version=None, system_prompt=None, user_prompt=None, example=None, input_data=None, deployment_name = None): 
    model = lm()
    model.set_client(endpoint, api_key, api_version)
    model.set_prompt(system_prompt=system_prompt, user_prompt=user_prompt, example=example)
    input_data["response"] = input_data["description"].apply(model.get_response, deployment_name=deployment_name) 
    return input_data

endpoint="https://oai-test-ss.openai.azure.com/"
api_key=""
api_version="2024-05-01-preview"
deployment_name = "gpt-35-turbo"
example = "Example: The book is about 11 year old Harry Potter, who receives a letter saying that he is invited to attend Hogwarts, school of witchcraft and wizardry. He then learns that a powerful wizard and his minions are after the sorcerer's stone that will make this evil wizard immortal and undefeatable. Answer: Fantasy"
system_prompt = "You are a helpful assistant. For each call, use provided example to perform task in precise manner. Return only the answer."
user_prompt = "Classify the book into a genre based on the description provided."

In [15]:
input_data = pd.read_csv(os.path.join(os.getcwd(),"book_desc.csv"))

In [16]:
output_df

NameError: name 'output_df' is not defined

In [17]:
model = SASAzureOpenAILLM()
model.set_client(endpoint="https://oai-test-ss.openai.azure.com/", api_key="", api_version="2024-05-01-preview")
model.set_prompt(system_prompt="You are a helpful assistant. For each call, use provided example to answer a provided question (Question:) in a concise manner.  Return only the answer.", user_prompt="Echo whatever is provided as context.", example="Example: The capital of France is Paris. Answer: The capital of Fance is Paris.")
# Test 1: Test single call with context (Meaning the cell is not empty)
model.get_response(context="The capital of France is Paris.", deployment_name="gpt-35-turbo")



'The capital of France is Paris.'

In [18]:
# Test 2: Test single call with no context (Meaning the cell is empty)
model.get_response(deployment_name="gpt-35-turbo")

No context provided


''

In [19]:
# Test 3: Test single call with explicit deployment name
model.deployment_name = "gpt-35-turbo"
model.get_response(context="The capital of France is Paris.")

'The capital of France is Paris.'

In [20]:
input_data

,title,author,description
0,Onyx Storm,Rebecca Yarros,After nearly eighteen months at Basgiath War C...
1,Beautiful Ugly,Alice Feeney,Author Grady Green is having the worst best da...
2,Great Big Beautiful Life,Emily Henry,Alice Scott is an eternal optimist still dream...


In [23]:
# Test 4: Test multiple calls from table; all with context 
example = "Example: The book is about 11 year old Harry Potter, who receives a letter saying that he is invited to attend Hogwarts, school of witchcraft and wizardry. He then learns that a powerful wizard and his minions are after the sorcerer's stone that will make this evil wizard immortal and undefeatable. Answer: Fantasy"
system_prompt = "You are a helpful assistant. For each call, use provided example to perform task in precise manner. Return only the answer."
user_prompt = "Classify the book into a genre based on the description provided."
api_key = ""
def execute(): 
    model = SASAzureOpenAILLM()
    model.set_client(endpoint=endpoint, api_key=api_key, api_version=api_version)
    model.set_prompt(system_prompt=system_prompt, user_prompt=user_prompt, example=example)
    input_data["response"] = input_data["description"].apply(model.get_response, deployment_name="gpt-35-turbo") 
    # df = df.apply(lambda x: np.square(x) if x.name == 'd' else x, axis=1)
    # result = input_data["response"] = input_data.apply(lambda x: model.get_response(context= x['description'], deployment_name = "gpt-35-turbo" ))
   
execute()


In [24]:
input_data

,title,author,description,response
0,Onyx Storm,Rebecca Yarros,After nearly eighteen months at Basgiath War C...,Answer: Fantasy.
1,Beautiful Ugly,Alice Feeney,Author Grady Green is having the worst best da...,Genre: Thriller/Mystery
2,Great Big Beautiful Life,Emily Henry,Alice Scott is an eternal optimist still dream...,Answer: Fiction


In [25]:
model.get_prompt()

'You are a helpful assistant. For each call, use provided example to answer a provided question (Question:) in a concise manner.  Return only the answer.Echo whatever is provided as context.\nExample(s): Example: The capital of France is Paris. Answer: The capital of Fance is Paris.\nContext: The capital of France is Paris.Context: The capital of France is Paris.'

In [26]:
reset_indf()

In [27]:
# Test 5: Test multiple calls from table; some with no context
# Adding new row without a description
# new_row = {'title': 'The Crash', 'author': "Frieda McFadden", 'description': ''}
# input_data.loc[len(input_data)] = new_row
execute()
print(input_data)
# reset_indf()



No context provided
                      title           author  \
0                Onyx Storm   Rebecca Yarros   
1            Beautiful Ugly     Alice Feeney   
2  Great Big Beautiful Life      Emily Henry   
3                 The Crash  Frieda McFadden   

                                         description  \
0  After nearly eighteen months at Basgiath War C...   
1  Author Grady Green is having the worst best da...   
2  Alice Scott is an eternal optimist still dream...   
3                                                      

                                            response  
0                                    Genre: Fantasy.  
1                            Genre: Mystery/Thriller  
2  Answer: Fiction (could be classified as a myst...  
3                                                     


In [28]:
input_data

,title,author,description,response
0,Onyx Storm,Rebecca Yarros,After nearly eighteen months at Basgiath War C...,Genre: Fantasy.
1,Beautiful Ugly,Alice Feeney,Author Grady Green is having the worst best da...,Genre: Mystery/Thriller
2,Great Big Beautiful Life,Emily Henry,Alice Scott is an eternal optimist still dream...,Answer: Fiction (could be classified as a myst...
3,The Crash,Frieda McFadden,,


In [29]:
# Test 6: Test multiple calls from table; all with no examples aka zero-shot prompting

In [30]:
model.set_prompt(system_prompt="You are a helpful assistant. For each call, use provided example to answer a provided question (Question:) in a concise manner.  Return only the answer.", user_prompt="Echo whatever is provided as context.", example="Example: The capital of France is Paris. Answer: The capital of Fance is Paris.")

### Previous Functions

In [31]:
def get_client(endpoint = None, api_key = None, api_version = None):
    from openai import AzureOpenAI
    import os
    if endpoint is None:
        endpoint = os.environ["AZURE_OPENAI_ENDPOINT"] 
    if api_key is None:
        api_key = os.environ["AZURE_OPENAI_API_KEY"] 
    if api_version is None:
        api_version = os.environ["AZURE_OPENAI_API_VERSION"]

    client = AzureOpenAI( api_key = api_key,  api_version = api_version, azure_endpoint = endpoint)
    return client


In [32]:
def get_prompt(system_prompt = None, user_prompt = None, example = None, context = None):
    if system_prompt is None:
        system_prompt = "You are a helpful assistant. For each call, use provided context (Context:) to answer a provided question (Question:) in a concise manner.  Return only the answer."
    if user_prompt is None:
        user_prompt = "Echo whatever is provided as context."
    if context is None:
        context = "Echo"
    if example is None: 
        return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f"Question: {user_prompt} \n Context: {context}" 
        }
        ]
    else:
        return [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": f"Question: {user_prompt}. \n Example(s): {example} \n Context: {context}" 
        }
        ]


In [33]:
def call_llm(prompt = None, client = None, deployment = None, temperature = None):
    if prompt is None:
        prompt = get_prompt()
    if client is None:
        client = get_client()
    if deployment is None:
        deployment = os.environ["DEPLOYMENT_NAME"]
    if temperature is None:
        temperature = 0.7
    completion = client.chat.completions.create(
        model = deployment,
        messages = prompt,
        temperature = temperature
    )  
    return completion.choices[0].message.content

In [34]:
def execute(context = None, user_prompt=None, system_prompt= None):
    prompt = get_prompt(context = context, user_prompt = user_prompt, system_prompt = system_prompt)
    client = get_client()
    return call_llm(prompt = prompt, client = client)


In [35]:
import pandas as pd
input_data = pd.read_csv("book_desc.csv", skipinitialspace=True)

print(input_data.shape)

import os
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://oai-test-ss.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["DEPLOYMENT_NAME"] = "gpt-35-turbo"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-10-21"

user_prompt = "Provide a summary of this book in one sentence."
system_prompt = "You are a helpful library assistant who categorises books or summarises them, or provides recommendations."

input_data["response"] = input_data["description"].apply(execute, user_prompt = user_prompt, system_prompt = system_prompt)

(3, 3)


APIConnectionError: Connection error.